- Change the np.load to include the file
- Add the following .pkl files into the same folder as this .ipynb (model, model_features, scaler, poly)
- Edit the transform_features() method if necessary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy
import joblib
import networkx as nx

test=np.load("../datasets/labeled/third_batch_multi.npz")

X_test=test["X"]

XX_test = pd.DataFrame(X_test)
XX_test.rename(columns={0:"user",1:"item",2:"rating"},inplace=True)

num_unique_users = XX_test["user"].nunique()
print(f"Number of unique user IDs in the test set: {num_unique_users}")

Number of unique user IDs in the test set: 1100


In [2]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
from sklearn.preprocessing import StandardScaler

EPS = 1e-6

def transform_features(X):
    """
    Transforms the input DataFrame X (assumed to have columns 'user', 'item', 'rating')
    into a DataFrame with aggregated user-level features. These include:
      - Aggregation of counts and summary statistics.
      - Proportional features.
      - Gap statistics on item IDs.
      - Min, max, median, and variance of movie IDs.
      - Product features based on item and rating.
      - Rating distribution entropy.
      - Movie popularity features.
      - Unique movies count and diversity.
      - Deviation from population average rating.
      - Sequential pattern features (rating differences, etc.).
      - Advanced interaction features and review count binning.

    Returns a DataFrame with all the computed features.
    """
    df = X.copy()

    # --- AGGREGATION: Compute counts and summary statistics ---
    user_features = df.groupby("user").agg(
        review_count=("rating", "count"),
        avg_rating=("rating", "mean"),
        std_rating=("rating", "std"),
        like_count=("rating", lambda x: (x == 10).sum()),
        dislike_count=("rating", lambda x: (x == -10).sum()),
        unknown_count=("rating", lambda x: (x == 1).sum()),
        neutral_count=("rating", lambda x: (x == 0).sum()),
    ).reset_index()

    # --- PROPORTIONAL FEATURES ---
    user_features["like_pct"] = user_features["like_count"] / (user_features["review_count"] + EPS)
    user_features["dislike_pct"] = user_features["dislike_count"] / (user_features["review_count"] + EPS)
    user_features["unknown_pct"] = user_features["unknown_count"] / (user_features["review_count"] + EPS)
    user_features["neutral_pct"] = user_features["neutral_count"] / (user_features["review_count"] + EPS)

    # --- GAP STATISTICS ---
    XX_sorted = X.sort_values(by=['user', 'item'])
    XX_sorted['item_diff'] = XX_sorted.groupby('user')['item'].diff().fillna(0)
    gap_stats = XX_sorted.groupby('user')['item_diff'].agg(['mean', 'std', 'max', 'min'])
    gap_stats.columns = ['gap_mean', 'gap_std', 'gap_max', 'gap_min']
    user_features = user_features.merge(gap_stats, on='user', how='left')

    # --- MIN/MAX/MEDIAN/VARIANCE OF MOVIE IDs ---
    min_max_df = X.groupby("user")["item"].agg(min_movie="min", max_movie="max", median_movie="median", variance_movie="var").reset_index()
    user_features = user_features.merge(min_max_df, on="user", how="left")

    # --- PRODUCT FEATURES ---
    X = X.copy()  # work on a copy to avoid modifying original data
    X["item_rating"] = X["item"] * X["rating"]
    sum_rating = X.groupby("user")["rating"].sum().reset_index(name="sum_rating")
    sum_product = X.groupby("user")["item_rating"].sum().reset_index(name="sum_item_rating")
    user_features = user_features.merge(sum_product, on="user", how="left")
    user_features = user_features.merge(sum_rating, on="user", how="left")

    user_features["average_product"] = user_features["sum_item_rating"] / user_features["review_count"]
    user_features["product_above_zero"] = (user_features["sum_item_rating"] > 0).astype(int)
    user_features["sum_above_zero"] = (user_features["sum_rating"] > 0).astype(int)
    user_features["avg_product_vs_avg_rating"] = user_features["average_product"] / (user_features["avg_rating"] + EPS)

    # --- RATING DISTRIBUTION ENTROPY ---
    def calc_entropy(row):
        probs = [row["like_pct"], row["dislike_pct"], row["unknown_pct"], row["neutral_pct"]]
        probs = [p for p in probs if p > 0]
        return entropy(probs) if probs else 0
    user_features["rating_entropy"] = user_features.apply(calc_entropy, axis=1)

    # --- MOVIE POPULARITY FEATURES ---
    movie_popularity = X.groupby("item").size().reset_index(name="movie_popularity")
    X_with_pop = X.merge(movie_popularity, on="item")
    pop_features = X_with_pop.groupby("user").agg(
        avg_movie_popularity=("movie_popularity", "mean"),
        std_movie_popularity=("movie_popularity", "std"),
        min_movie_popularity=("movie_popularity", "min"),
        max_movie_popularity=("movie_popularity", "max"),
    ).reset_index()

    # --- UNIQUE MOVIES AND DIVERSITY ---
    unique_items = X.groupby("user")["item"].nunique().reset_index()
    unique_items.columns = ["user", "unique_movies"]

    # --- DEVIATION FROM POPULATION FEATURES ---
    movie_avg_rating = X.groupby("item")["rating"].mean().reset_index(name="movie_avg_rating")
    X_with_avg = X.merge(movie_avg_rating, on="item")
    X_with_avg["rating_deviation"] = X_with_avg["rating"] - X_with_avg["movie_avg_rating"]
    X_with_avg["abs_rating_deviation"] = np.abs(X_with_avg["rating_deviation"])
    deviation_features = X_with_avg.groupby("user").agg(
        mean_deviation=("rating_deviation", "mean"),
        std_deviation=("rating_deviation", "std"),
        mean_abs_deviation=("abs_rating_deviation", "mean"),
        max_abs_deviation=("abs_rating_deviation", "max"),
    ).reset_index()

    # --- SEQUENTIAL PATTERN FEATURES ---
    X_sorted = X.sort_values(["user", "item"])
    X_sorted["next_rating"] = X_sorted.groupby("user")["rating"].shift(-1)
    X_sorted["rating_diff"] = X_sorted["next_rating"] - X_sorted["rating"]
    X_sorted["abs_rating_diff"] = np.abs(X_sorted["rating_diff"])
    X_sorted = X_sorted.dropna(subset=["rating_diff"])
    sequence_features = X_sorted.groupby("user").agg(
        mean_rating_diff=("rating_diff", "mean"),
        std_rating_diff=("rating_diff", "std"),
        mean_abs_rating_diff=("abs_rating_diff", "mean"),
        max_abs_rating_diff=("abs_rating_diff", "max"),
        rating_changes_count=("rating_diff", lambda x: (x != 0).sum()),
    ).reset_index()
    sequence_features["rating_changes_pct"] = sequence_features["rating_changes_count"] / (
        user_features.set_index("user")["review_count"] - 1 + EPS
    ).reindex(sequence_features["user"]).values

    # --- COMBINE ALL FEATURES ---
    all_features = user_features.merge(pop_features, on="user", how="left")
    all_features = all_features.merge(unique_items, on="user", how="left")
    all_features = all_features.merge(deviation_features, on="user", how="left")
    all_features = all_features.merge(sequence_features, on="user", how="left")

    all_features = all_features.fillna(0)

    # --- UNSUPERVISED ANOMALY DETECTION FEATURES ---
    feature_cols = [col for col in all_features.columns
                    if col not in ["user", "label", "is_anomalous"]
                    and all_features[col].dtype in [np.float64, np.int64]]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(all_features[feature_cols])

    # --- ADVANCED INTERACTION FEATURES ---
    all_features["like_dislike_ratio"] = all_features["like_count"] / (all_features["dislike_count"] + EPS)
    all_features["rating_range"] = all_features["max_abs_rating_diff"]
    all_features["popularity_vs_deviation"] = all_features["avg_movie_popularity"] * all_features["mean_abs_deviation"]
    all_features["entropy_by_count"] = all_features["rating_entropy"] * np.log1p(all_features["review_count"])

    # --- BINNING FEATURES ---
    all_features["review_count_bin"] = pd.qcut(all_features["review_count"], q=5, labels=False, duplicates="drop")

    return pd.DataFrame(all_features)

In [3]:
# Feature engineering
test_features = transform_features(XX_test)
test_features 
test_features.sort_values(by="user", inplace=True) #Sort by user

# Select only important features
model_features = joblib.load("model_features.pkl")

print(f"test_features before selecting features {test_features.shape}")

# If feature does not exist, populate with 0s
for feat in model_features:
    if feat not in test_features.columns:
        test_features[feat] = 0
test_features = test_features[model_features]

print(f"test_features after selecting features {test_features.shape}")


print(test_features.columns)
print(test_features.head())

# Load scaler and polynomial transformer, and transform the test features
scaler = joblib.load("scaler.pkl")
poly = joblib.load("poly.pkl")
test_features_scaled = scaler.transform(test_features)
test_features_poly = poly.transform(test_features_scaled)

# Load the trained model and predict probabilities (shape: #test_users x 6)
xgb_model = joblib.load("model.pkl")
probabilities = xgb_model.predict_proba(test_features_poly)
y_pred_proba_rf = xgb_model.predict_proba(test_features_poly)
print(y_pred_proba_rf)


import joblib

# Save the predictions as an .npz file
np.savez("predictions.npz", probabilities=probabilities)
print(f"prediction shape {probabilities.shape}")

# View predictions.npz
test_results=np.load("predictions.npz")
test_results_df = pd.DataFrame(test_results["probabilities"])
print(test_results_df.head())

test_features before selecting features (1100, 47)
test_features after selecting features (1100, 35)
Index(['review_count', 'avg_rating', 'std_rating', 'like_count',
       'dislike_count', 'neutral_count', 'dislike_pct', 'unknown_pct',
       'neutral_pct', 'rating_entropy', 'avg_movie_popularity',
       'std_movie_popularity', 'min_movie_popularity', 'max_movie_popularity',
       'std_deviation', 'mean_abs_deviation', 'max_abs_deviation',
       'mean_rating_diff', 'std_rating_diff', 'max_abs_rating_diff',
       'rating_changes_pct', 'like_dislike_ratio', 'popularity_vs_deviation',
       'entropy_by_count', 'min_movie', 'max_movie', 'median_movie',
       'variance_movie', 'sum_item_rating', 'average_product',
       'product_above_zero', 'sum_above_zero', 'avg_product_vs_avg_rating',
       'gap_mean', 'gap_max'],
      dtype='object')
   review_count  avg_rating  std_rating  like_count  dislike_count  \
0           213    3.408451    5.135643          73              7   
1    

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PolynomialFeatures from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:29:56] WARNI

In [4]:
data = np.load('predictions.npz')
predictions = data['probabilities']  # Assuming the array is stored under 'arr_0'

class_counts = {i: 0 for i in range(6)}  # Initialize counts for each class

for row in predictions:
    predicted_class = np.argmax(row)  # Get the class with the highest probability
    class_counts[predicted_class] += 1

print("Class instance counts:")
for class_label, count in class_counts.items():
    print(f"Class {class_label}: {count}") 

Class instance counts:
Class 0: 972
Class 1: 16
Class 2: 14
Class 3: 25
Class 4: 34
Class 5: 39


In [5]:
from sklearn.metrics import roc_auc_score

y_true = test["y"]  # True labels

# Convert true labels to a DataFrame
df_y_true = pd.DataFrame(y_true, columns=["user", "true_label"])

# Load the predicted probabilities
predictions_data = np.load("predictions.npz")
probabilities = predictions_data["probabilities"]

# Get the predicted class (argmax of each row)
predicted_labels = np.argmax(probabilities, axis=1)

# Create a DataFrame for predictions
df_predictions = pd.DataFrame({
    "user": df_y_true["user"],  # Assuming user IDs match in order
    "true_label": df_y_true["true_label"],
    "predicted_label": predicted_labels
})

# Identify misclassified users
df_predictions["correct"] = df_predictions["true_label"] == df_predictions["predicted_label"]
df_misclassified = df_predictions[df_predictions["correct"] == False]
# Display the comparison
# tools.display_dataframe_to_user(name="True vs Predicted Labels", dataframe=df_predictions)
df_misclassified
df_misclassified.to_csv("misclassified_users.csv", index=False)

# Compute AUC for each class
auc_per_class = {}
for i in range(probabilities.shape[1]):  # Iterate over all classes
    binary_true = (df_predictions["true_label"] == i).astype(int)
    try:
        auc = roc_auc_score(binary_true, probabilities[:, i])
        auc_per_class[i] = auc
        print(f"  Class {i}: AUC = {auc:.3f}")
    except ValueError:
        auc_per_class[i] = None  # If AUC cannot be computed (e.g., only one class present)

k = 5
AUC_0 = auc_per_class[0]
anomaly_aucs = [auc_per_class[i] for i in range(1, k+1) if i in auc_per_class]

final_metric = (0.5 * AUC_0) + (0.5 / k) * sum(anomaly_aucs)
print(f"\n🏆 Final Evaluation Metric: {final_metric:.3f}")

# Convert AUC scores to DataFrame
df_auc = pd.DataFrame(list(auc_per_class.items()), columns=["class", "AUC"])


  Class 0: AUC = 0.861
  Class 1: AUC = 0.756
  Class 2: AUC = 0.834
  Class 3: AUC = 0.975
  Class 4: AUC = 0.766
  Class 5: AUC = 0.699

🏆 Final Evaluation Metric: 0.834
